In [64]:
import numpy as np
import KwikSort as kwik
import utils as ul
import pandas as pd
from time import time
import metrics

In [65]:
df=pd.read_csv('20230918dataset_uf.csv')
#add a ranking all m candidates has smaller rank than w's
df["ranker 11"]=np.arange(1,101,1)
#add a ranking half  m candidates on the top of ranking and another half m candidates on the bottom of rinking and w candidates on the middle  
ranker_12=np.concatenate((np.arange(1,16,1),np.arange(86,101,1),np.arange(16,86,1)), axis=None)
#print(ranker_12)
df["ranker 12"]=ranker_12
df=df.drop(['ranker 9','ranker 10'],axis=1)
#df=df[df.columns[8:11]]
df

,Unnamed: 0,ranker 1,ranker 2,ranker 3,ranker 4,ranker 5,ranker 6,ranker 7,ranker 8,ranker 11,ranker 12
0,M01,20,86,79,32,28,70,21,71,1,1
1,M02,14,9,7,52,55,26,89,72,2,2
2,M03,22,3,24,31,97,29,41,73,3,3
3,M04,54,87,13,22,95,46,32,74,4,4
4,M05,93,6,53,92,31,10,56,75,5,5
...,...,...,...,...,...,...,...,...,...,...,...
95,W66,44,29,85,15,72,11,67,66,96,81
96,W67,50,66,29,83,4,81,40,67,97,82
97,W68,65,75,78,80,68,12,73,68,98,83
98,W69,41,60,9,34,99,30,63,69,99,84


In [66]:
rankings=df[df.columns[1:]].to_numpy()
rankings

array([[ 20,  86,  79,  32,  28,  70,  21,  71,   1,   1],
       [ 14,   9,   7,  52,  55,  26,  89,  72,   2,   2],
       [ 22,   3,  24,  31,  97,  29,  41,  73,   3,   3],
       [ 54,  87,  13,  22,  95,  46,  32,  74,   4,   4],
       [ 93,   6,  53,  92,  31,  10,  56,  75,   5,   5],
       [ 24,  38,  88,   2, 100,  61,  68,  76,   6,   6],
       [ 10,  45,  31,  59,  98,  84,  80,  77,   7,   7],
       [ 85,  37,  77,  76,  19,   3,  50,  78,   8,   8],
       [ 59,  70,  44,  53,  27,  54,  49,  79,   9,   9],
       [ 31,  97,  97,  25,  77,   1,  16,  80,  10,  10],
       [ 58,  90,   5,   6,  18,  67,   8,  81,  11,  11],
       [ 12,  49,  90,  44,  67,   6,   2,  82,  12,  12],
       [ 47,  63,  60,  69,  32,  15,  19,  83,  13,  13],
       [ 86,  61,  43,  49,  37,  63,  97,  84,  14,  14],
       [ 21,  69,  49,  63,   8,  65,  57,  85,  15,  15],
       [ 17,  46,  34,  97,  91,  36,  91,  86,  16,  86],
       [ 71,  76,  70,  64,  39,  24,  14,  87,  17,  87

In [67]:
rankings=rankings.transpose()
rankings,rankings.shape

(array([[ 20,  14,  22,  54,  93,  24,  10,  85,  59,  31,  58,  12,  47,
          86,  21,  17,  71,   2,  94,  76,  36,   6,  56,  43,  19,  48,
          64,  33,  52,  23,  16,  11,  99,  38,  92,  84,  90,  75,  30,
          97,  13,  78,   5,  42,  62,   4,  88,  40,  15,  68,  29,  87,
          91,  46,  72,  74,  57,  25,  49,   7,  18,  66,  35,  60,  28,
          45,  83,  77,  80,   1,   9,  89,  81,  39,  73,  37,  27,  53,
          98,  34,  55,  69,  67,  82,  96,  79,  63, 100,  95,   3,  32,
          26,   8,  61,  51,  44,  50,  65,  41,  70],
        [ 86,   9,   3,  87,   6,  38,  45,  37,  70,  97,  90,  49,  63,
          61,  69,  46,  76,  74,  15,  80,  16,  17,  95,  89,   4,   5,
          58,  36,  59,  33,  39,  30,  28,  77,  26,  91,  47,  52,  88,
          24,   2, 100,  44,  92,  40,  22,  14,  72,  42,  13,  41,  18,
          10,  82,  57,  48,  27,  85,   7,  98,  68,  65,  54,  94,  64,
          35,  79,  81,  19,  73,  12,  96,  78,  50,  34

In [68]:
threshold=0.1

In [69]:
attribute = np.append(np.repeat(0, 30), np.repeat(1, 70))
attribute=np.c_[np.arange(100),attribute]
df_groups=pd.DataFrame(attribute,columns=['id','attribute'])
groups=df_groups.groupby(df_groups.columns[1]).count()
groups=groups.to_numpy()
groups=np.c_[np.array([[0],[1]]),groups]
Vertices=attribute[:,0]
Vertices



array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [70]:
kemeny_ranking_Kwik=kwik.KwikSort(Vertices,rankings)

In [71]:
kemeny_ranking_Kwik


array([ 1, 46, 20, 31, 45, 10, 93,  0, 11, 12, 30, 47, 40, 42, 44, 48, 60,
       34, 43, 57, 38, 63,  9, 73,  2, 76, 26, 29,  3, 23, 54, 94, 24,  8,
       13, 14, 75, 80, 82, 49, 50, 35, 37, 41, 58, 52, 32, 69, 53, 55, 56,
       62, 64, 65, 66, 70, 77, 81, 72, 78, 84, 86, 51, 89,  4, 61, 74, 95,
       99, 88, 92,  5, 59,  6,  7, 36, 17, 33, 39, 85, 68, 83, 87, 90, 91,
       79, 98, 96, 21, 67, 15, 16, 22, 25, 27, 28, 97, 19, 71, 18])

In [72]:
kemeny_dist_kwik=ul.kemeny_dist(rankings,kemeny_ranking_Kwik)
kemeny_dist_kwik

24104

In [73]:
t0=time()
fair_Kwik_Sort_ranking=kwik.fair_Kwik_Sort(rankings,attribute,groups,threshold)
t1=time()   
t1-t0

10.712010145187378

In [74]:
fair_kemeny_dist_kwik=ul.kemeny_dist(rankings,fair_Kwik_Sort_ranking)

In [75]:
fair_kemeny_dist_kwik,fair_Kwik_Sort_ranking

(23910,
 array([ 1, 31, 45, 56, 20,  5, 40, 42, 26, 30, 47, 43, 55,  3, 10, 44, 46,
        48, 60, 58, 52, 92, 93, 11, 94, 24, 50, 29, 54, 62, 64,  8, 14, 34,
        53, 57, 38, 12, 63, 49, 61, 16, 65, 66,  2, 69, 70,  4, 76, 89,  0,
        73, 81, 88, 91,  7, 95, 35,  9, 98, 37, 80, 75, 82, 78, 13, 84, 86,
        17, 51, 96, 99, 18, 90, 19, 97, 21,  6, 23, 32, 74, 36, 79, 33, 39,
        67, 77, 15, 72, 22, 83, 25, 27, 85, 41, 59, 68, 28, 71, 87]))

In [76]:
fair_Kwik_Sort_ranking_satisfied_array=[]
k_threshold=threshold
for k in range(1,100):
    fair_Kwik_Sort_ranking_k =pd.DataFrame(columns=['id', 'ranker'])
    fair_Kwik_Sort_ranking_k["id"]=np.arange(0,100,1)
    fair_Kwik_Sort_ranking_k["ranker"]=fair_Kwik_Sort_ranking
    fair_Kwik_Sort_ranking_k_m =fair_Kwik_Sort_ranking_k.head(30).to_numpy()
    fair_Kwik_Sort_ranking_k_w =fair_Kwik_Sort_ranking_k.tail(70).to_numpy()
    fair_Kwik_Sort_ranking_k_groups=np.array([fair_Kwik_Sort_ranking_k_m,fair_Kwik_Sort_ranking_k_w], dtype=object)
    fair_Kwik_Sort_ranking_k_satisfied=metrics.top_k_parity(fair_Kwik_Sort_ranking_k_groups,k,k_threshold )
    fair_Kwik_Sort_ranking_satisfied_array.append(["ranker",k,fair_Kwik_Sort_ranking_k_satisfied])

In [77]:
fair_Kwik_Sort_ranking_satisfied_array

[['ranker', 1, True],
 ['ranker', 2, True],
 ['ranker', 3, True],
 ['ranker', 4, True],
 ['ranker', 5, True],
 ['ranker', 6, True],
 ['ranker', 7, True],
 ['ranker', 8, True],
 ['ranker', 9, True],
 ['ranker', 10, True],
 ['ranker', 11, True],
 ['ranker', 12, True],
 ['ranker', 13, True],
 ['ranker', 14, True],
 ['ranker', 15, True],
 ['ranker', 16, True],
 ['ranker', 17, True],
 ['ranker', 18, True],
 ['ranker', 19, True],
 ['ranker', 20, True],
 ['ranker', 21, True],
 ['ranker', 22, True],
 ['ranker', 23, True],
 ['ranker', 24, True],
 ['ranker', 25, True],
 ['ranker', 26, True],
 ['ranker', 27, True],
 ['ranker', 28, True],
 ['ranker', 29, True],
 ['ranker', 30, True],
 ['ranker', 31, True],
 ['ranker', 32, True],
 ['ranker', 33, True],
 ['ranker', 34, True],
 ['ranker', 35, True],
 ['ranker', 36, True],
 ['ranker', 37, True],
 ['ranker', 38, True],
 ['ranker', 39, True],
 ['ranker', 40, True],
 ['ranker', 41, True],
 ['ranker', 42, True],
 ['ranker', 43, True],
 ['ranker', 44, True

In [78]:
fair_Kwik_Sort_ranking_m=np.c_[np.arange(30),fair_Kwik_Sort_ranking[:30]]
fair_Kwik_Sort_ranking_w=np.c_[np.arange(30,100),fair_Kwik_Sort_ranking[30:]]

In [79]:
fair_Kwik_Sort_ranking_parity=metrics.pairwise_statistical_parity(fair_Kwik_Sort_ranking_m,fair_Kwik_Sort_ranking_w)
fair_Kwik_Sort_ranking_parity

0.2038095238095238

In [15]:
base_rankings = np.load('cs_baseranks.npy').astype(int)
base_rankings

array([[ 0,  1, 11, ..., 61, 56, 49],
       [ 0,  3,  1, ..., 60, 52, 61],
       [ 0,  3,  5, ..., 43, 49, 52],
       ...,
       [ 0,  2,  8, ..., 52, 61, 53],
       [ 0,  1,  4, ..., 63, 52, 61],
       [ 0,  2,  1, ..., 61, 62, 53]])

In [140]:


t0=time()
kemeny_ranking_Kwik_cs=kwik.KwikSort(base_rankings)
t1=time()
t1-t0

[] [1] []
[1] [3] []
[] [0] [1 3]
[] [5] []
[5] [6] []
[] [4] [5 6]
[0 1 3] [2] [4 5 6]
[] [8] []
[8] [9] []
[8 9] [10] []
[ 8  9 10] [11] []
[0 1 3 2 4 5 6] [7] [ 8  9 10 11]
[] [13] []
[] [18] []
[13] [14] [18]
[] [15] []
[15] [19] []
[] [24] [15 19]
[] [23] []
[24 15 19] [16] [23]
[] [26] []
[] [20] [26]
[20 26] [28] []
[] [25] []
[] [29] []
[] [30] [29]
[25] [27] [30 29]
[20 26 28] [22] [25 27 30 29]
[] [35] []
[] [31] []
[35] [36] [31]
[] [41] []
[] [46] []
[41] [34] [46]
[] [33] [41 34 46]
[35 36 31] [37] [33 41 34 46]
[20 26 28 22 25 27 30 29] [32] [35 36 31 37 33 41 34 46]
[24 15 19 16 23] [21] [20 26 28 22 25 27 30 29 32 35 36 31 37 33 41 34 46]
[13 14 18] [17] [24 15 19 16 23 21 20 26 28 22 25 27 30 29 32 35 36 31 37 33 41 34 46]
[ 0  1  3  2  4  5  6  7  8  9 10 11] [12] [13 14 18 17 24 15 19 16 23 21 20 26 28 22 25 27 30 29 32 35 36 31 37 33
 41 34 46]
[] [38] []
[] [43] []
[43] [40] []
[43 40] [49] []
[43 40 49] [47] []
[38] [42] [43 40 49 47]
[38 42 43 40 49 47] [52] []
[

In [141]:
kemeny_ranking_Kwik_cs

array([ 0,  1,  3,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 17,
       24, 15, 19, 16, 23, 21, 20, 26, 28, 22, 25, 27, 30, 29, 32, 35, 36,
       31, 37, 33, 41, 34, 46, 39, 38, 42, 43, 40, 49, 47, 52, 45, 44, 48,
       51, 50, 53, 54, 55, 64, 60, 61, 56, 57, 63, 59, 58, 62])

In [142]:
kemeny_dist_kwik_cs=ul.kemeny_dist(base_rankings,kemeny_ranking_Kwik_cs)
kemeny_dist_kwik_cs

5713

In [144]:
length_L=random.randint(0,99)
length_R=random.randint(0,99-length_L)
print(length_L,length_R)



31 53


In [145]:
Vertices_R=[]
Vertices_L=[]
Vertices=list(range(0,100))
print(Vertices)
for i in range(length_L):
    vertice=Vertices[random.randint(0,len(Vertices)-1)]
    Vertices_L.append(vertice)
    Vertices.remove(vertice)
for i in range(length_R):
    vertice=Vertices[random.randint(0,len(Vertices)-1)]
    Vertices_R.append(vertice)
    Vertices.remove(vertice)   
pivot_candidate=[Vertices[random.randint(0,len(Vertices)-1)]]
Vertices_R,Vertices_L,pivot_candidate

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


([37,
  58,
  42,
  86,
  57,
  81,
  78,
  6,
  40,
  77,
  83,
  95,
  91,
  33,
  89,
  96,
  54,
  24,
  4,
  32,
  30,
  39,
  12,
  18,
  16,
  97,
  1,
  68,
  49,
  0,
  69,
  34,
  25,
  72,
  44,
  64,
  76,
  51,
  8,
  13,
  23,
  46,
  99,
  50,
  26,
  67,
  43,
  52,
  41,
  2,
  62,
  47,
  7],
 [71,
  65,
  35,
  19,
  15,
  85,
  17,
  55,
  80,
  21,
  92,
  63,
  61,
  27,
  36,
  45,
  20,
  14,
  59,
  3,
  5,
  31,
  9,
  38,
  87,
  90,
  73,
  11,
  66,
  60,
  10],
 [98])

In [146]:

pivot_candidate,pivot_candidate[0]

([98], 98)

In [255]:
def fair_Kwik_Sort_recur(unweighted_majority_Tournament,Fas_Tournament,attribute,groups, threshold):
    Vertices=unweighted_majority_Tournament[0]
    Arc=unweighted_majority_Tournament[1]
    size_candidates=len(Vertices)
    if size_candidates ==0:
        print(np.zeros(0),'null')
        return np.zeros(0)

    pivot_candidate=Vertices[randrange(size_candidates)]
    pivot_candidate=[pivot_candidate]
    if size_candidates==1:
        print(pivot_candidate[0],'pi')
        return np.array([pivot_candidate[0]])
    Vertices_L=[]
    Vertices_R=[]
    for j in range(size_candidates):
        if Arc[Vertices[j],pivot_candidate[0]]==1:
            Vertices_L.append(Vertices[j])
        if Arc[pivot_candidate[0],Vertices[j]]==1:
            Vertices_R.append(Vertices[j])

    if size_candidates==2:
        result=np.concatenate((Vertices_L,np.array([pivot_candidate[0]]).astype(int),Vertices_R))
        print(result,'re')
        return result
    
    Vertices_L, pivot_candidate, Vertices_R=adjust_unfairness(rankings,Vertices_L,pivot_candidate,Vertices_R,groups,attribute,threshold)
    print(Vertices_L,pivot_candidate,Vertices_R,'addver')
    unweighted_majority_Tournament_L=generate_unweighted_majority_Tournament(Vertices_L,Fas_Tournament)
    unweighted_majority_Tournament_R=generate_unweighted_majority_Tournament(Vertices_R,Fas_Tournament)
    
    permutation_L=fair_Kwik_Sort_recur(unweighted_majority_Tournament_L,Fas_Tournament,attribute,groups,threshold)
    permutation_R=fair_Kwik_Sort_recur(unweighted_majority_Tournament_R,Fas_Tournament,attribute,groups,threshold)
    if len(permutation_L)>0:
        permutation_L=permutation_L.astype(int)
    if len(permutation_R)>0:
         permutation_R=permutation_R.astype(int)
    if len(pivot_candidate)>0:
        print(permutation_L,pivot_candidate[0],permutation_R,'per1')
        return np.concatenate((permutation_L,np.array([pivot_candidate[0]]).astype(int),permutation_R))
    else:
        print(permutation_L,permutation_R,'per2')
        return np.concatenate((permutation_L,permutation_R))

In [256]:
def fair_Kwik_Sort(rankings,attribute,groups,threshold):
    Vertices=np.arange(rankings.shape[1])
    Fas_Tournament=build_graph(rankings)
    unweighted_majority_Tournament=generate_unweighted_majority_Tournament(Vertices,Fas_Tournament)
    ranking_opt=fair_Kwik_Sort_recur(rankings,unweighted_majority_Tournament,Fas_Tournament,attribute,groups,threshold)

    return ranking_opt

In [257]:



from math import ceil
from numpy.ma import floor

def boundary(proportion_group,length_Vertices,threshold):
    return(round((proportion_group-threshold)*(length_Vertices)),round((proportion_group+threshold)*(length_Vertices)))
    
def adjust_unfairness(rankings,Vertices_L,pivot_candidate,Vertices_R,groups,attribute,threshold):
    length_Vertices_L=len(Vertices_L)
    length_Vertices_R=len(Vertices_R)
    #print(length_Vertices_L,length_Vertices_R,pivot_candidate)
    proportions_groups=proportions(groups)
    boundary_L=[]
    boundary_L.append(boundary(proportions_groups[0],length_Vertices_L,threshold))
    boundary_L.append(boundary(proportions_groups[1],length_Vertices_L,threshold))
    boundary_R=[]
    boundary_R.append(boundary(proportions_groups[0],length_Vertices_R,threshold))
    boundary_R.append(boundary(proportions_groups[1],length_Vertices_R,threshold))
    #print(boundary_L,boundary_R,'boundary')
    groups_of_Vertices_L=group_by(Vertices_L,groups,attribute)
    groups_of_Vertices_R=group_by(Vertices_R,groups,attribute)
    size_of_group_L=(len(groups_of_Vertices_L[0]),len(groups_of_Vertices_L[1]))
    size_of_group_R=(len(groups_of_Vertices_R[0]),len(groups_of_Vertices_R[1]))
    #print(size_of_group_L,size_of_group_R,'size')
    unfairness_of_group_L=[]
    unfairness_of_group_L.append((max(0,boundary_L[0][0]-size_of_group_L[0]),max(0,size_of_group_L[0]-boundary_L[0][1])))
    unfairness_of_group_L.append((max(0,boundary_L[1][0]-size_of_group_L[1]),max(0,size_of_group_L[1]-boundary_L[1][1])))
    unfairness_of_group_R=[]
    unfairness_of_group_R.append((max(0,boundary_R[0][0]-size_of_group_R[0]),max(0,size_of_group_R[0]-boundary_R[0][1])))
    unfairness_of_group_R.append((max(0,boundary_R[1][0]-size_of_group_R[1]),max(0,size_of_group_R[1]-boundary_R[1][1])))
    #print(unfairness_of_group_L,unfairness_of_group_R,'unfair')
    group_pivot=attribute[pivot_candidate[0]][1]
    for group in groups:
       # print(group,'p')
        if group_pivot==group[0]:
            #print(group,group_pivot,'0p')
            gap_fairness_pivot=np.zeros(4)
            gap_fairness_pivot[0]=unfairness_of_group_L[group[0]][0]
            gap_fairness_pivot[1]=unfairness_of_group_R[group[0]][0]
            gap_fairness_pivot[2]=unfairness_of_group_L[group[0]][1]
            gap_fairness_pivot[3]=unfairness_of_group_R[group[0]][1]
            max_unfairness_pivot=np.argmax(gap_fairness_pivot)
            if gap_fairness_pivot[max_unfairness_pivot] >0:
                if max_unfairness_pivot==0:
                    kemeny_ranking_R=KwikSort(groups_of_Vertices_R[group[0]],rankings)
                    Vertices_L.append(pivot_candidate[0])
                    size_of_group_L_pivot=size_of_group_L[group[0]]+1
                    new_boundary_L_pivot=boundary(proportions_groups[group[0]],len(Vertices_L),threshold)[0]
                    for i in range(int(gap_fairness_pivot[max_unfairness_pivot] )-1):
                        #print(i,size_of_group_L_pivot,new_boundary_L_pivot,'a')
                        if size_of_group_L_pivot>=new_boundary_L_pivot:
                            break
                        Vertices_L.append(kemeny_ranking_R[i])
                        Vertices_R.remove(kemeny_ranking_R[i])
                        size_of_group_L_pivot=size_of_group_L_pivot+1
                        new_boundary_L_pivot=boundary(proportions_groups[group[0]],len(Vertices_L),threshold)[0]
                    pivot_candidate=[]
                
                if max_unfairness_pivot==1:
                    kemeny_ranking_L=KwikSort(groups_of_Vertices_L[group[0]],rankings)
                    Vertices_R.append(pivot_candidate[0])
                    size_of_group_R_pivot=size_of_group_L[group[0]]
                    new_boundary_R_pivot=boundary(proportions_groups[group[0]],len(Vertices_R),threshold)[0]
                    size_of_kemeny_ranking_L = len(kemeny_ranking_L)
                    for i in range(int(gap_fairness_pivot[max_unfairness_pivot] )):
                        #print(i,size_of_group_R_pivot,new_boundary_R_pivot,size_of_kemeny_ranking_L,'b')
                        if size_of_group_R_pivot>=new_boundary_R_pivot:
                            break
                        if(size_of_kemeny_ranking_L==0):
                            break
                        Vertices_R.append(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        Vertices_L.remove(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        size_of_group_R_pivot=size_of_group_R_pivot+1
                        new_boundary_R_pivot=boundary(proportions_groups[group[0]],len(Vertices_R),threshold)[0]
                    pivot_candidate=[]
                
                if max_unfairness_pivot==2:
                    kemeny_ranking_L=KwikSort(groups_of_Vertices_L[group[0]],rankings)
                    Vertices_R.append(pivot_candidate[0])
                    size_of_group_L_pivot=size_of_group_L[group[0]]
                    new_boundary_L_pivot=boundary(proportions_groups[group[0]],len(Vertices_L),threshold)[1]
                    size_of_kemeny_ranking_L = len(kemeny_ranking_L)
                    for i in range(int(gap_fairness_pivot[max_unfairness_pivot] )):
                        #print(i,size_of_group_L_pivot,new_boundary_L_pivot,size_of_kemeny_ranking_L,'c')
                        if size_of_group_L_pivot<=new_boundary_L_pivot:
                            break
                        if(size_of_kemeny_ranking_L==0):
                            break
                        Vertices_R.append(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        Vertices_L.remove(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        size_of_group_L_pivot=size_of_group_L_pivot-1
                        new_boundary_L_pivot=boundary(proportions_groups[group[0]],len(Vertices_L),threshold)[1]
                    pivot_candidate=[]
                    
                if max_unfairness_pivot==3:
                    kemeny_ranking_R=KwikSort(groups_of_Vertices_R[group[0]],rankings)
                    Vertices_L.append(pivot_candidate[0])
                    size_of_group_R_pivot=size_of_group_R[group[0]]
                    new_boundary_R_pivot=boundary(proportions_groups[group[0]],len(Vertices_R),threshold)[1]
                    for i in range(int(gap_fairness_pivot[max_unfairness_pivot] )):
                        #print(i,size_of_group_R_pivot,new_boundary_R_pivot,'d')
                        if size_of_group_R_pivot<=new_boundary_R_pivot:
                            break
                        Vertices_L.append(kemeny_ranking_R[i])
                        Vertices_R.remove(kemeny_ranking_R[i])
                        size_of_group_R_pivot-=1
                        new_boundary_R_pivot=boundary(proportions_groups[group[0]],len(Vertices_R),threshold)[1]
                    pivot_candidate=[]
  
        else:
            gap_fairness_non_pivot=np.zeros(4)
            gap_fairness_non_pivot[0]=unfairness_of_group_L[group[0]][0]
            gap_fairness_non_pivot[1]=unfairness_of_group_R[group[0]][0]
            gap_fairness_non_pivot[2]=unfairness_of_group_L[group[0]][1]
            gap_fairness_non_pivot[3]=unfairness_of_group_R[group[0]][1]
            max_unfairness_non_pivot=np.argmax(gap_fairness_non_pivot)
            
            if max_unfairness_non_pivot>=0:
                
                if gap_fairness_non_pivot[max_unfairness_non_pivot]==0:
                    kemeny_ranking_R=KwikSort(groups_of_Vertices_R[group[0]],rankings)
                    size_of_group_L_non_pivot=size_of_group_L[group[0]]
                    for i in range(int(gap_fairness_non_pivot[max_unfairness_non_pivot])):
                        #print(i,size_of_group_L_non_pivot,kemeny_ranking_R,'e')
                        Vertices_L.append(kemeny_ranking_R[i])
                        Vertices_R.remove(kemeny_ranking_R[i])
                        size_of_group_L_non_pivot+=1
                        new_boundary_L_non_pivot=boundary(proportions_groups[group[0]],len(Vertices_L),threshold)[0]
                        if size_of_group_L_non_pivot >= new_boundary_L_non_pivot:
                            break

            
                if max_unfairness_non_pivot==1:
                    kemeny_ranking_L=KwikSort(groups_of_Vertices_L[group[0]],rankings)
                    size_of_group_R_non_pivot=size_of_group_L[group[0]]
                    size_of_kemeny_ranking_L = len(kemeny_ranking_L)
                    for i in range(int(gap_fairness_non_pivot[max_unfairness_non_pivot])):
                       # print(i,size_of_group_R_non_pivot,size_of_kemeny_ranking_L,'f')
                        if(size_of_kemeny_ranking_L==0):
                            break
                        Vertices_R.append(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        Vertices_L.remove(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        size_of_group_R_non_pivot=size_of_group_R_non_pivot+1
                        new_boundary_R_non_pivot=boundary(proportions_groups[group[0]],len(Vertices_R),threshold)[0]
                       # print(i,size_of_group_R_non_pivot,size_of_kemeny_ranking_L,new_boundary_R_non_pivot,'f')
                        if size_of_group_R_non_pivot>=new_boundary_R_non_pivot:
                            break
            
                        
            
            
                if max_unfairness_non_pivot==2:
                    kemeny_ranking_L=KwikSort(groups_of_Vertices_L[group[0]],rankings)
                    size_of_group_L_non_pivot=size_of_group_L[group[0]]
                    size_of_kemeny_ranking_L = len(kemeny_ranking_L)
                    for i in range(int(gap_fairness_non_pivot[max_unfairness_non_pivot])):
                        size_of_kemeny_ranking_L = len(kemeny_ranking_L)
                        #print(i,size_of_group_L_non_pivot,size_of_kemeny_ranking_L,'g')
                        if(size_of_kemeny_ranking_L==0):#only 1 element situation
                            break
                        Vertices_R.append(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        Vertices_L.remove(kemeny_ranking_L[size_of_kemeny_ranking_L-i-1])
                        new_boundary_L_non_pivot=boundary(proportions_groups[group[0]],len(Vertices_L),threshold)[1]
                        #print(i,size_of_group_L_non_pivot,new_boundary_L_non_pivot,size_of_kemeny_ranking_L,'g')
                        if size_of_group_L_non_pivot <= new_boundary_L_non_pivot:
                            break


           
                if max_unfairness_non_pivot==3:
                    kemeny_ranking_R=KwikSort(groups_of_Vertices_R[group[0]],rankings)
                    size_of_group_R_non_pivot=size_of_group_R[group[0]]
                    for i in range(int(gap_fairness_non_pivot[max_unfairness_non_pivot])):
                        #print(i,size_of_group_R_non_pivot,'h')
                        Vertices_L.append(kemeny_ranking_R[i])
                        Vertices_R.remove(kemeny_ranking_R[i])
                        size_of_group_R_non_pivot-=1
                        new_boundary_R_non_pivot=boundary(proportions_groups[group[0]],len(Vertices_R),threshold)[1]
                        if size_of_group_R_non_pivot<=new_boundary_R_non_pivot:
                            break
             
 
    
    return Vertices_L, pivot_candidate, Vertices_R

def group_by(Vertices,groups,attribute):
    groups_by=[]
    for values in groups:
        group=[]
        groups_by.append(group)
    for vertex in Vertices:
        group_of_Vertice=attribute[vertex]
        groups_by[group_of_Vertice[1]].append(vertex)
    return groups_by


def proportions(groups):
    proportions = []
    total_number = np.sum(groups[:, 1])
    for group in groups:
        proportions.append(group[1] / total_number)
    proportions = np.array(proportions)
    return proportions

In [265]:
cs_groups_df=pd.read_csv('cs_groups.csv', header=None,names=['candidate','attr_1','attr_2'])
cs_groups_df=cs_groups_df[['candidate','attr_2']]
attribute=cs_groups_df.to_numpy()
groups=cs_groups_df.groupby(cs_groups_df.columns[1]).count()
groups=groups.to_numpy()
groups=np.c_[np.array([[0],[1]]),groups]
attribute,groups

(array([[ 0,  0],
        [ 1,  0],
        [ 2,  1],
        [ 3,  1],
        [ 4,  0],
        [ 5,  1],
        [ 6,  1],
        [ 7,  1],
        [ 8,  0],
        [ 9,  1],
        [10,  0],
        [11,  0],
        [12,  0],
        [13,  1],
        [14,  0],
        [15,  1],
        [16,  1],
        [17,  1],
        [18,  0],
        [19,  1],
        [20,  0],
        [21,  0],
        [22,  1],
        [23,  1],
        [24,  0],
        [25,  1],
        [26,  1],
        [27,  1],
        [28,  0],
        [29,  1],
        [30,  0],
        [31,  1],
        [32,  1],
        [33,  1],
        [34,  0],
        [35,  0],
        [36,  1],
        [37,  1],
        [38,  1],
        [39,  1],
        [40,  1],
        [41,  0],
        [42,  1],
        [43,  1],
        [44,  1],
        [45,  0],
        [46,  0],
        [47,  0],
        [48,  1],
        [49,  0],
        [50,  0],
        [51,  1],
        [52,  1],
        [53,  1],
        [54,  1],
        [5

In [266]:
t0=time()
fair_Kwik_Sort_ranking_cs=fair_Kwik_Sort(base_rankings,attribute,groups,threshold)
t1=time()
t1-t0

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51] [50] [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64] addver
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 24] [16] [18, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51] addver
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [14] [15, 17, 19, 20, 24] addver
[0, 1, 2, 3, 4, 5, 6, 10] [] [8, 9, 11, 12, 13, 7] addver
[0, 1, 3, 2] [] [5, 6, 10, 4] addver
[0, 1, 3, 2] [] [] addver
[] [0] [1, 3, 2] addver
[] null
[] [3] [2, 1] addver
[] null
[1. 2.] re
[] 3 [1 2] per1
[] 0 [3 1 2] per1
[] null
[0 3 1 2] [] per2
[5, 6, 4] [10] [] addver
[5, 4] [6] [] addver
[4. 5.] re
[] null
[4 5] 6 [] per1
[] null
[4 5 6] 10 [] per1
[0 3 1 2] [ 4  5  6 10] per2
[8, 9, 7] [11] [12, 13] addver
[] [7] [8, 9] addver
[] null
[8. 9.

In [267]:
fair_Kwik_Sort_ranking,fair_Kwik_Sort_ranking_cs

(array([10, 43, 11, 34, 38, 40,  1, 45, 42, 63, 46, 30, 47,  2, 20, 44, 48,
        55, 60,  0, 57,  8,  9, 24, 73, 58, 76, 81, 93,  3, 89, 31,  7, 99,
        91,  4, 54,  5, 12, 13, 62, 88, 92, 21, 49, 36, 37, 64, 50, 52, 70,
        14, 66, 69, 65, 35, 75, 53, 56, 61, 26, 39, 68, 77, 32, 72, 74, 29,
        78, 86, 95, 90, 79, 15, 94,  6, 16, 98, 23, 80, 51, 96, 17, 33, 82,
        19, 67, 85, 84, 22, 59, 41, 71, 87, 18, 25, 83, 27, 28, 97]),
 array([ 0,  3,  1,  2,  4,  5,  6, 10,  7,  8,  9, 11, 12, 13, 14, 17, 24,
        15, 20, 19, 16, 23, 18, 22, 21, 25, 27, 26, 30, 29, 32, 28, 31, 37,
        33, 36, 41, 38, 34, 35, 39, 42, 43, 40, 45, 46, 44, 48, 47, 49, 51,
        50, 53, 52, 54, 55, 57, 64, 61, 56, 60, 58, 62, 63, 59]))

In [268]:
fair_kemeny_dist_kwik=ul.kemeny_dist(rankings,fair_Kwik_Sort_ranking)
fair_kemeny_dist_kwik_cs=ul.kemeny_dist(base_rankings,fair_Kwik_Sort_ranking_cs)
fair_kemeny_dist_kwik,fair_kemeny_dist_kwik_cs

(23880, 5639)

In [395]:
result=adjust_unfairness(rankings,Vertices_L,[pivot_candidate],Vertices_R,groups,attribute,threshold)

[(9.0, 9), (21.0, 21)] [(12.0, 13), (28.0, 29)] boundary
(7, 23) (14, 27) size
[(2.0, 0), (0, 2)] [(0, 1), (1.0, 0)] unfair
1 46 gp
[ 0 30] 1p
[20, 19, 28, 8, 1, 15, 23, 12, 4, 13, 7, 26, 6, 14] [ 1  4  6 15 20 26  7  8 12 13 14 23 28 19] d
[ 1 70] 0p
0 lower
46 wtf2
2 upper


IndexError: list index out of range

In [329]:
result

([8,
  38,
  63,
  1,
  6,
  48,
  9,
  86,
  18,
  82,
  93,
  75,
  33,
  92,
  60,
  81,
  59,
  54,
  14,
  49,
  73,
  16,
  13,
  30,
  85,
  34,
  29,
  99,
  61,
  44],
 [],
 [7,
  42,
  26,
  70,
  0,
  57,
  74,
  12,
  98,
  24,
  95,
  94,
  25,
  53,
  65,
  76,
  3,
  89,
  51,
  41,
  97,
  69,
  15,
  79,
  40,
  2,
  91,
  78,
  68,
  32,
  96,
  66,
  77,
  80,
  67,
  55,
  90,
  50,
  58,
  35,
  36,
  22])

In [17]:
def proportions(groups):
    proportions = []
    total_number = np.sum(groups[:, 1])
    for group in groups:
        proportions.append(group[1] / total_number)
    proportions = np.array(proportions)
    return proportions

In [18]:
from random import randrange
from itertools import combinations


def generate_unweighted_majority_Tournament(Verticles,Fas_Tournament):
    size_candidates=Fas_Tournament.shape[0]
    size_Vertices=len(Verticles)
    Arc=np.zeros((size_candidates,size_candidates))
    for i,j in combinations(range(size_Vertices), 2):
        if Fas_Tournament[Verticles[i],Verticles[j]]>Fas_Tournament[Verticles[j],Verticles[i]]:
            Arc[Verticles[i],Verticles[j]]=1
        else:
            Arc[Verticles[j],Verticles[i]]=1
    return Verticles,Arc



def Kwik_Sort_recur(unweighted_majority_Tournament,Fas_Tournament):
    Vertices=unweighted_majority_Tournament[0]
    Arc=unweighted_majority_Tournament[1]
    if len(Vertices) ==0:
        return np.zeros(0)
    size_candidates=len(Vertices)
    pivot_candidate=Vertices[randrange(size_candidates)]
    Vertices_L=[]
    Vertices_R=[]
    for j in range(size_candidates):
        if Arc[Vertices[j],pivot_candidate]==1:
            Vertices_L.append(Vertices[j])
        if Arc[pivot_candidate,Vertices[j]]==1:
            Vertices_R.append(Vertices[j])
    Vertices_L=np.array(Vertices_L)
    Vertices_R=np.array(Vertices_R)
    unweighted_majority_Tournament_L=generate_unweighted_majority_Tournament(Vertices_L,Fas_Tournament)
    unweighted_majority_Tournament_R=generate_unweighted_majority_Tournament(Vertices_R,Fas_Tournament)
    permutation_L=Kwik_Sort_recur(unweighted_majority_Tournament_L,Fas_Tournament).astype(int)
    permutation_R=Kwik_Sort_recur(unweighted_majority_Tournament_R,Fas_Tournament).astype(int)
    return np.concatenate((permutation_L,np.array([pivot_candidate]).astype(int),permutation_R))


def KwikSort(Vertices,rankings):
    Fas_Tournament=build_graph(rankings)
    unweighted_majority_Tournament=generate_unweighted_majority_Tournament(Vertices,Fas_Tournament)
    ranking_opt=Kwik_Sort_recur(unweighted_majority_Tournament,Fas_Tournament)

    return ranking_opt

def build_graph(rankings):
    n_voters, size_candidate = rankings.shape
    edge_weights = np.zeros((size_candidate, size_candidate))
    for i, j in combinations(range(size_candidate), 2):
        preference = rankings[:, i] - rankings[:, j]
        preference_i_j = np.sum(preference < 0)  # prefers i to j
        preference_j_i = np.sum(preference > 0)  # prefers j to i
        if preference_i_j > preference_j_i:
            edge_weights[i, j] = preference_i_j
        elif preference_i_j < preference_j_i:
            edge_weights[j, i] = preference_j_i
    return edge_weights